<a href="https://colab.research.google.com/github/weimarquintero/UDEA-ai4eng-20242/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Creando el kaggle.json para descargar los datos directamente desde Kaggle.

In [14]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

udea-ai4eng-20242.zip: Skipping, found more recently modified local copy (use --force to force download)


Descomprimiendo el fichero y contando el numero de lineas de cada archivo.

In [38]:
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [39]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  50135751 test.csv
   692501  10666231 118025055 train.csv
  1286075  15528571 172877479 total


Importando las librerias necesarias para trabajar.

In [52]:
import pandas as pd
import numpy as np

Cargando el archivo train.csv con pandas

In [53]:
df_train = pd.read_csv("train.csv")
print ("shape of loaded dataframe", df_train.shape)
df_train.head()

shape of loaded dataframe (692500, 12)


,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo


Validando las columnas de nuestro DF.

In [54]:
df_train.columns

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'ESTU_PAGOMATRICULAPROPIO',
       'RENDIMIENTO_GLOBAL'],
      dtype='object')

Seleccionamos las columnas que definimos como candidatas para la creación del modelo en la fase de exploración.

In [55]:
df_train = df_train[['ESTU_VALORMATRICULAUNIVERSIDAD','ESTU_HORASSEMANATRABAJA','FAMI_TIENEINTERNET','RENDIMIENTO_GLOBAL']]
df_train.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_TIENEINTERNET,RENDIMIENTO_GLOBAL
0,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Si,medio-alto
1,Entre 2.5 millones y menos de 4 millones,0,No,bajo
2,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Si,bajo
3,Entre 4 millones y menos de 5.5 millones,0,Si,alto
4,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Si,medio-bajo


Validando los valores faltantes en cada una de estas columnas.

In [56]:
sum(df_train.ESTU_VALORMATRICULAUNIVERSIDAD.isna()), sum(df_train.ESTU_HORASSEMANATRABAJA.isna()), sum(df_train.FAMI_TIENEINTERNET.isna())

(6287, 30857, 26629)

Sustituimos los valores faltantes con la etiqueta 'no info'.

In [57]:
df_train.ESTU_VALORMATRICULAUNIVERSIDAD.values[df_train.ESTU_VALORMATRICULAUNIVERSIDAD.isna()] = 'no info'
df_train.ESTU_HORASSEMANATRABAJA.values[df_train.ESTU_HORASSEMANATRABAJA.isna()] = 'no info'
df_train.FAMI_TIENEINTERNET.values[df_train.FAMI_TIENEINTERNET.isna()] = 'no info'

In [58]:
sum(df_train.ESTU_VALORMATRICULAUNIVERSIDAD.isna()), sum(df_train.ESTU_HORASSEMANATRABAJA.isna()), sum(df_train.FAMI_TIENEINTERNET.isna())

(0, 0, 0)

**Limpiando 'ESTU_VALORMATRICULAUNIVERSIDAD'.**

In [59]:
cmap = {'Entre 1 millón y menos de 2.5 millones': 1.75,
 'Entre 2.5 millones y menos de 4 millones': 3.25,
 'Menos de 500 mil': .250,
 'Entre 500 mil y menos de 1 millón': .75,
 'Entre 4 millones y menos de 5.5 millones': 4.75,
 'Más de 7 millones': 7.75,
 'Entre 5.5 millones y menos de 7 millones': 6.25,
 'No pagó matrícula': 0,
 'no info': -1}

df_train.ESTU_VALORMATRICULAUNIVERSIDAD = np.r_[[cmap[i] for i in df_train.ESTU_VALORMATRICULAUNIVERSIDAD]]
df_train.ESTU_VALORMATRICULAUNIVERSIDAD.value_counts()

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
1.75,204048
3.25,127430
0.25,80263
0.75,78704
4.75,69736
7.75,68014
6.25,38490
0.00,19528
-1.00,6287


**Limpiando 'ESTU_HORASSEMANATRABAJA'.**

In [60]:
cmap = {'Entre 11 y 20 horas': 1.5,
 'Entre 21 y 30 horas': 2.5,
 'Menos de 10 horas': .5,
 'Más de 30 horas': 3.5,
 '0': 0,
 'no info': -1}

df_train.ESTU_HORASSEMANATRABAJA = np.r_[[cmap[i] for i in df_train.ESTU_HORASSEMANATRABAJA]]
df_train.ESTU_HORASSEMANATRABAJA.value_counts()

,count
ESTU_HORASSEMANATRABAJA,
3.5,249352
0.0,116550
1.5,115857
2.5,92693
0.5,87191
-1.0,30857


**Limpiando 'FAMI_TIENEINTERNET'.**

In [61]:
cmap = {'Si': 1,
 'No': 0,
 'no info': -1}

df_train.FAMI_TIENEINTERNET = np.r_[[cmap[i] for i in df_train.FAMI_TIENEINTERNET]]
df_train.FAMI_TIENEINTERNET.value_counts()

,count
FAMI_TIENEINTERNET,
1,592514
0,73357
-1,26629


**Convirtiendo la columna de la predicción a valores discretos.**

In [62]:
y_col = 'RENDIMIENTO_GLOBAL'

rmap = {'alto': 3, 'bajo':0, 'medio-bajo':1, 'medio-alto':2}
df_train[y_col] = [rmap[i] for i in df_train[y_col]]
df_train.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_TIENEINTERNET,RENDIMIENTO_GLOBAL
0,6.25,0.5,1,2
1,3.25,0.0,0,0
2,3.25,3.5,1,0
3,4.75,0.0,1,3
4,3.25,2.5,1,1
